In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=13779e8b2a3dcde694f7a65caaaed75a59c489c4dad4b1c6ffdc47a9be3804d8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
# create the context
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2023-01-09 09:13:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  8.18MB/s    in 1.6s    

2023-01-09 09:13:29 (8.18 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://d95c-34-73-183-144.ngrok.io


In [5]:
from google.colab import files
files.upload()

Saving heroes_information.csv to heroes_information.csv


{'heroes_information.csv': b",name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight\r\n0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0\r\n1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0\r\n2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0\r\n3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0\r\n4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0\r\n5,Absorbing Man,Male,blue,Human,No Hair,193.0,Marvel Comics,-,bad,122.0\r\n6,Adam Monroe,Male,blue,-,Blond,-99.0,NBC - Heroes,-,good,-99.0\r\n7,Adam Strange,Male,blue,Human,Blond,185.0,DC Comics,-,good,88.0\r\n8,Agent 13,Female,blue,-,Blond,173.0,Marvel Comics,-,good,61.0\r\n9,Agent Bob,Male,brown,Human,Brown,178.0,Marvel Comics,-,good,81.0\r\n10,Agent Zero,Male,-,-,-,191.0,Marvel Comics,-,good,104.0\r\n11,Air-Walker,Male,blue,-,White,188.0,Marvel Comics,-,bad,108.0\r\n12,Ajax,Male,brown,Cy

In [6]:
df = spark.read.csv('heroes_information.csv', header='True')
df.show(10)

+---+-------------+------+---------+-----------------+----------+------+-----------------+----------+---------+------+
|_c0|         name|Gender|Eye color|             Race|Hair color|Height|        Publisher|Skin color|Alignment|Weight|
+---+-------------+------+---------+-----------------+----------+------+-----------------+----------+---------+------+
|  0|       A-Bomb|  Male|   yellow|            Human|   No Hair| 203.0|    Marvel Comics|         -|     good| 441.0|
|  1|   Abe Sapien|  Male|     blue|    Icthyo Sapien|   No Hair| 191.0|Dark Horse Comics|      blue|     good|  65.0|
|  2|     Abin Sur|  Male|     blue|          Ungaran|   No Hair| 185.0|        DC Comics|       red|     good|  90.0|
|  3|  Abomination|  Male|    green|Human / Radiation|   No Hair| 203.0|    Marvel Comics|         -|      bad| 441.0|
|  4|      Abraxas|  Male|     blue|    Cosmic Entity|     Black| -99.0|    Marvel Comics|         -|      bad| -99.0|
|  5|Absorbing Man|  Male|     blue|            

In [7]:
df.filter(df.Gender=='Male').count()

505

In [8]:
df.filter(df.Gender=='Female').count()

200

In [10]:
df.groupby('Race').count().show()

+------------------+-----+
|              Race|count|
+------------------+-----+
|          Neyaphem|    1|
|          Symbiote|    9|
|        Kryptonian|    7|
|           Ungaran|    1|
|           Martian|    1|
|         Strontian|    1|
| Human / Radiation|   11|
|   Kakarantharaian|    1|
|         Metahuman|    2|
|Dathomirian Zabrak|    1|
|    Human / Cosmic|    2|
|   Xenomorph XX121|    1|
|            Animal|    4|
|       Frost Giant|    2|
|            Cyborg|   11|
|           Inhuman|    4|
|     God / Eternal|   14|
|    Yoda's species|    1|
|     Icthyo Sapien|    1|
|            Rodian|    1|
+------------------+-----+
only showing top 20 rows

